<a href="https://colab.research.google.com/github/felixzhao/title_catgories_classification/blob/main/Job_title_classification_V2.2_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this work, 
I need to preprocess the raw data by following steps

1. merge data.csv and categories.csv, create a raw dataset
2. preprocess the "x_title"

    1. remove stopwords
    2. punctuations
    3. stemming
    4. lemmatiztion

3. save to raw.csv for following step.

# install dependency

In [14]:
! pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=d6df1c6bf2b12ac3e872b4779fb544ec8a3b7091437e0129fc4d5f5bd80bf3e6
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


# install dependency

In [15]:
import numpy as np
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [16]:
# need to install stopwords and some other package by following code
# input "d" to download view, then input package name to start package download
import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [17]:
from sentence_transformers import SentenceTransformer, util

# load data

load data
This work loads data from google drive. Please put the files into "trademe_data" folder under the root of google drive.

Then run the following code to mount the google drive in colab.

You can follow the guide. https://towardsdatascience.com/different-ways-to-connect-google-drive-to-a-google-colab-notebook-pt-1-de03433d2f7a

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
root_path = "drive/MyDrive/trademe_data/"

In [42]:
cat_raw = pd.read_csv(f"{root_path}categories.csv")
print(len(cat_raw))
cat_raw.head(1)

196


,y_cat_id,cat_1,cat_2,cat_3
0,4957,construction & roading,health & safety,health & safety


In [43]:
cat_raw[cat_raw.duplicated()]

,y_cat_id,cat_1,cat_2,cat_3


In [44]:
data_raw = pd.read_csv(f"{root_path}data.csv")
print(len(data_raw))
data_raw.head(1)

20000


,x_title,y_cat_id
0,unqualified asbestos remover,5192


In [45]:
raw_df = pd.merge(data_raw, cat_raw, on='y_cat_id', how='left')
print(len(raw_df))
raw_df.head(1)

20000


,x_title,y_cat_id,cat_1,cat_2,cat_3
0,unqualified asbestos remover,5192,trades & services,labourers,labourers


# functions

In [24]:
# translate_table = dict((ord(char), ' ') for char in string.punctuation)
# def remove_punctuation(s):
#     return s.translate(translate_table)

In [25]:
# def remove_stopwords_punctuation(s):
#     stop = set(stopwords.words('english') + list(string.punctuation))
#     return ' '.join([i for i in word_tokenize(s.lower()) if i not in stop])

In [26]:

stop = set(stopwords.words('english') + list(string.punctuation))
# Initialize Python porter stemmer
ps = PorterStemmer()
# Initialize wordnet lemmatizer
wnl = WordNetLemmatizer()

def sentence_preprocess(sentence, stemming=False, lemmatization=True):
    # remove stopwords & punctuation
    words = [i for i in word_tokenize(sentence.lower()) if i not in stop]
    if stemming:
        words = [ps.stem(i) for i in words]
    if lemmatization:
        words = [wnl.lemmatize(word, pos="v") for word in words]
    return ' '.join(words)


# preprocess function analysis

from following output,
we can see if stemming only then it will cutoff some word, make it unreadable
but lemmatization only will not tans the word to root.

in this work, we use lemma only for now

these code reference from
https://www.datacamp.com/tutorial/stemming-lemmatization-python

In [ ]:
example_sentence = "Python programmers often tend like programming in python because it's like english. We call people who program in python pythonistas."


In [ ]:
sentence_preprocess(example_sentence, stemming=True, lemmatization=False)

"python programm often tend like program python 's like english call peopl program python pythonista"

In [ ]:
sentence_preprocess(example_sentence, stemming=False, lemmatization=True)

"python programmers often tend like program python 's like english call people program python pythonistas"

In [ ]:
sentence_preprocess(example_sentence, stemming=True, lemmatization=True)

"python programm often tend like program python 's like english call peopl program python pythonista"

# Preprocess

In [39]:
raw_output_path = f"{root_path}raw.csv"
raw_title_encoding_path =f"{root_path}raw_title_encoding.csv"
raw_title_feature_encoding_path =f"{root_path}raw_title_feature_encoding.csv"

# proprecss x_title column

In [46]:
raw_df['x_title']

0                           unqualified asbestos remover
1                                    senior test analyst
2                               ict trainer / supervisor
3        automotive specialists *multi faceted position*
4                                       business analyst
                              ...                       
19995                                    quality control
19996           showroom consultant | project specialist
19997                                civil site engineer
19998                                  senior scaffolder
19999              registered mental health professional
Name: x_title, Length: 20000, dtype: object

In [47]:
raw_df['x_title_feature'] = raw_df['x_title'].apply(sentence_preprocess)

# preprocess response col

In [48]:
raw_df['cats'] = raw_df.cat_1.str.cat(raw_df.cat_2, sep='-').str.cat(raw_df.cat_3, sep='-')

In [51]:
raw_df.head(2)

,x_title,y_cat_id,cat_1,cat_2,cat_3,x_title_feature,cats
0,unqualified asbestos remover,5192,trades & services,labourers,labourers,unqualified asbestos remover,trades & services-labourers-labourers
1,senior test analyst,5123,it,testing,testing,senior test analyst,it-testing-testing


In [50]:

raw_df.to_csv(raw_output_path)

# Add embedding

In [30]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [31]:
def get_embedding(text):
    if not text:
        print("Content is empty.")
        return None
    return model.encode(input, convert_to_tensor=True)

## title encoding

In [52]:
raw_title_encoding_df = raw_df.copy()

In [33]:
raw_title_encoding_df['ecoder_title'] = raw_title_encoding_df['x_title'].apply(lambda x: model.encode(x, convert_to_tensor=True))

In [36]:
raw_title_encoding_df.head(2)

,x_title,y_cat_id,cat_1,cat_2,cat_3,x_title_feature,cats,ecoder_title
0,unqualified asbestos remover,5192,trades & services,labourers,labourers,unqualified asbestos remover,trades & services-labourers-labourers,"[tensor(-0.0663), tensor(0.0821), tensor(0.047..."
1,senior test analyst,5123,it,testing,testing,senior test analyst,it-testing-testing,"[tensor(-0.0241), tensor(0.0556), tensor(-0.10..."


In [38]:
raw_df.to_csv(raw_title_encoding_path)

## encoding title feature

In [34]:
raw_df['ecoder_title_feature'] = raw_df['x_title_feature'].apply(lambda x: model.encode(x, convert_to_tensor=True))

KeyboardInterrupt: ignored

# Show result

In [ ]:
raw_df

,x_title,y_cat_id,cat_1,cat_2,cat_3,x_title_feature,cats
0,unqualified asbestos remover,5192,trades & services,labourers,labourers,unqualified asbestos remover,trades & services-labourers-labourers
1,senior test analyst,5123,it,testing,testing,senior test analyst,it-testing-testing
2,ict trainer / supervisor,6894,education,tutoring & training,tutoring & training,ict trainer supervisor,education-tutoring & training-tutoring & training
3,automotive specialists *multi faceted position*,5197,trades & services,technicians,technicians,automotive specialists multi faceted position,trades & services-technicians-technicians
4,business analyst,5114,it,business & systems analysts,business & systems analysts,business analyst,it-business & systems analysts-business & syst...
...,...,...,...,...,...,...,...
19995,quality control,5140,manufacturing & operations,quality assurance,quality assurance,quality control,manufacturing & operations-quality assurance-q...
19996,showroom consultant | project specialist,5167,sales,sales reps,sales reps,showroom consultant project specialist,sales-sales reps-sales reps
19997,civil site engineer,5058,engineering,civil & structural,civil & structural,civil site engineer,engineering-civil & structural-civil & structural
19998,senior scaffolder,5042,construction & roading,supervisors & forepersons,supervisors & forepersons,senior scaffolder,construction & roading-supervisors & foreperso...


# Save to files

In [ ]:
output_path = "raw.csv"
raw_df.to_csv(output_path)